# Constants

In [1]:
import os, json
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["AZURE_AIFSTDEASTUS_PROJECT_ENDPOINT"]
deployment_name =  "gpt-4.1" # os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = "2025-05-15-preview" # os.environ["OPENAI_API_VERSION"] # at least "2025-04-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://basicfdrymlbe.services.ai.azure.com/api/projects/project>
azure-ai-projects library installed version: 1.0.0b12
azure-ai-agents library installed version: 1.1.0b3


# [Create AI Foundry Agent Client](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview)
**Note**: `AIProjectClient` could be replaced by `AgentsClient`, which is easier to read. However, `Project SDK` is recommended.<br/>

Anyway --> `project_client.agents == agents_client`

In [2]:
from azure.ai.agents import AgentsClient
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

# [Announcing MCP Support in Azure AI Foundry Agent Service](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)
- [Microsoft Learn Docs MCP Server](https://github.com/MicrosoftDocs/mcp)
- [Connect to Model Context Protocol (MCP) Servers (Preview)](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)
- [How to use the Model Context Protocol (MCP) tool WITH http](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)
- [AI Foundry Basic Agent Setup](https://github.com/azure-ai-foundry/foundry-samples/tree/main/samples/microsoft/infrastructure-setup/40-basic-agent-setup)
- [Announcing Model Context Protocol Support (preview) in Azure AI Foundry Agent Service](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)
- Region: WestUS

aif02westusgrp
basicfdrydl5j
aif02basicproject01

In [3]:
# MCP Agent Instructions
agent_name = "MCP_learn_agent"

agent_instructions = "You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer " \
"queries about Microsoft Learn portal."

# Question
question = """Which Azure OpenAI models are being deprecated, and what can I use to replace them?
Please provide details for each model and version, highlighting the differences between versions.
Include deprecation dates and recommended alternative versions."""

In [4]:
# MCP Tool definition

mcp_docsearch_definition = {
    "type": "mcp", #il tool è di tipo MCP
    "server_label": "microsoft_docs_search", #il nome univoco per MCP Server di MSLearn è "microsoft_docs_search"
    "server_url": "https://learn.microsoft.com/api/mcp", #l'URL del server MCP di MSLearn
    # "require_approval": "never" 
    "allowed_tools":["fetch_generic_documentation", "search_generic_code", "search_generic_documentation"]
}
mcp_docsearch_definition

{'type': 'mcp',
 'server_label': 'microsoft_docs_search',
 'server_url': 'https://learn.microsoft.com/api/mcp',
 'allowed_tools': ['fetch_generic_documentation',
  'search_generic_code',
  'search_generic_documentation']}

# Agent creation

In [5]:
agent_id = "" # ex: asst_sKeyVdFbgk5JMwCmQ0it80I6. If provided, it will be loaded rather than created

if agent_id != "":
    agent = project_client.agents.get_agent(agent_id=agent_id)
else:
    agent = project_client.agents.create_agent(
        model=deployment_name,
        name=agent_name,
        instructions=agent_instructions,
        tools=[mcp_docsearch_definition], # leave this line commented to create a naked agent ;-)
        tool_resources=None,
        # headers={"x-ms-enable-preview": "true"}
    )

agent.as_dict()

{'id': 'asst_EA3Z6uHaV9ZQUglmwZ7uQ9vI',
 'object': 'assistant',
 'created_at': 1751498400,
 'name': 'MCP_learn_agent',
 'description': None,
 'model': 'gpt-4.1',
 'instructions': 'You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer queries about Microsoft Learn portal.',
 'tools': [{'type': 'mcp',
   'server_label': 'microsoft_docs_search',
   'server_url': 'https://learn.microsoft.com/api/mcp',
   'allowed_tools': ['fetch_generic_documentation',
    'search_generic_code',
    'search_generic_documentation']}],
 'top_p': 1.0,
 'temperature': 1.0,
 'tool_resources': {},
 'metadata': {},
 'response_format': 'auto'}

# Create the thread and attach a new message to it

In [6]:
# Create a thread
thread = project_client.agents.threads.create()
print(f"Created thread: {thread}\n")

Created thread: {'id': 'thread_eHTLK4i4PZSYqB28fjEKLQ4Z', 'object': 'thread', 'created_at': 1751498401, 'metadata': {}, 'tool_resources': {}}



In [7]:
# Add a user message to the thread
from azure.ai.agents.models import MessageRole

message = project_client.agents.messages.create(
    thread_id=thread.id, 
    role=MessageRole.USER, 
    content= question
)
print(f"Created message: {message}")

Created message: {'id': 'msg_VSdZpB9BB7hC2rMgY6ZTjZQn', 'object': 'thread.message', 'created_at': 1751498401, 'assistant_id': None, 'thread_id': 'thread_eHTLK4i4PZSYqB28fjEKLQ4Z', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Which Azure OpenAI models are being deprecated, and what can I use to replace them?\nPlease provide details for each model and version, highlighting the differences between versions.\nInclude deprecation dates and recommended alternative versions.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent syncrhonously

In [8]:
%%time

import time

run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress"]:
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")
    time.sleep(1) # Wait for a second

print()

if run.status == "failed":
    print(f"Run error: {run}")
else:
    run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Run step: {step.id}, status: {step.status}, type: {step.type}, step: {step}")
        if step.type == "tool_calls":
            print(f"Tool call details:")
            for tool_call in step.step_details.tool_calls:
                print(json.dumps(tool_call.as_dict(), indent=2))

Run status: RunStatus.QUEUED
Run status: RunStatus.QUEUED
Run status: RunStatus.QUEUED
Run status: RunStatus.QUEUED
Run status: RunStatus.QUEUED
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED

Run step: step_8QCcTySmgJqwCb38BhaVr0lk, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION, step: {'id': 'step_8QCcTySmgJqwCb38BhaVr0lk', 'object': 'thread.run.step', 'created_at': 1751498413, 'run_id': 'run_Fob64JoJlfEdIhXT7bvQLZyA', 'assistant_id': 'asst_EA3Z6uHaV9ZQUglmwZ7uQ9vI', 'thread_id': 'thread_eHTLK4i4PZSYqB28fjEKLQ4Z', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1751498419, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message

# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.agents.models import MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, MessageTextFilePathAnnotation
from IPython.display import display, Markdown

image_files = []
annotations = []
citations = []

if run.status == 'completed':
    messages = project_client.agents.messages.list(thread_id=thread.id)
    messages_list = list(project_client.agents.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) -->")
                display(Markdown(c.text.value))
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) -->


Which Azure OpenAI models are being deprecated, and what can I use to replace them?
Please provide details for each model and version, highlighting the differences between versions.
Include deprecation dates and recommended alternative versions.


===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) -->


Here is an overview based on current Microsoft and Azure OpenAI documentation (as of June 2024):

### Deprecated Azure OpenAI Models

#### 1. GPT-3 models
- **Models Deprecated:**
  - text-davinci-003
  - text-davinci-002
  - text-ada-001
  - text-babbage-001
  - text-curie-001
- **Deprecation Date:** January 10, 2024
- **Recommended Replacement:** GPT-3.5 models (especially gpt-35-turbo), and GPT-4 models for advanced use cases.
- **Differences:** GPT-3.5 and GPT-4 offer significantly improved natural language understanding, reasoning, and response quality. They tend to be more accurate and exhibit better conversation/context handling.

#### 2. GPT-3.5 Turbo (Preview) versions
- **Models Deprecated:**
  - gpt-35-turbo (Preview releases: 0301, 0613)
- **Deprecation Date:** March 2024
- **Recommended Replacement:** Use the stable gpt-35-turbo or migrate to GPT-4 (gpt-4, gpt-4-32k, or latest gpt-4-turbo).
- **Differences:** Newer GPT-3.5 turbo versions offer enhanced reliability and stability. GPT-4 models provide even more advanced language capabilities and support longer context windows.

#### 3. Codex Models
- **Models Deprecated:**
  - code-davinci-002
  - code-cushman-001, code-cushman-002
- **Deprecation Date:** March 23, 2024
- **Recommended Replacement:** gpt-35-turbo and gpt-4 models can now handle code tasks more efficiently and with more natural language context.
- **Differences:** The newer models have improved code understanding, can manage larger context, and understand code in conversation with other queries.

### Key Upgrade Recommendations

- For chat, text analysis, and code generation, upgrade to the latest gpt-35-turbo or gpt-4 (or gpt-4-turbo) models.
- Always specify the exact model and version to ensure you get updates and the latest improvements.
- Deprecations mean that after the published date, you will no longer be able to access those legacy models via the API.

### Summary Table

| Deprecated Model        | Deprecation Date | Replacement         | Main Differences                   |
|------------------------|------------------|---------------------|------------------------------------|
| GPT-3 (Davinci, etc.)  | Jan 10, 2024     | GPT-3.5, GPT-4      | Better reasoning, context, speed   |
| GPT-3.5 Turbo (Preview)| Mar 2024         | GPT-3.5 Turbo (Stable), GPT-4 | More features, stability        |
| Codex                  | Mar 23, 2024     | GPT-3.5 Turbo, GPT-4| Improved code support, NL context  |

If you need a detailed, model-by-model comparison table or programmable usage examples, let me know!

# Print annotations from the messages

In [10]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    project_client.agents.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of file path annotations: 0



# Fetch citations from the messages

In [11]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 0



# Retrieve and download eventual images

In [12]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    project_client.agents.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of image contents: 0



# Collect all resources for this project

In [13]:
all_agents = list_all_agents(client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=project_client)
print(all_threads["summary"])

all_files = list_all_files(client=project_client)
print(all_files["summary"])

all_runs = list_all_runs(client=project_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(project_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(project_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=project_client)
print(all_vectorstores["summary"])

2 agents
2 threads
0 files
2 runs in 2 threads
0 vector stores


# Teardown for all resources

In [14]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    project_client.agents.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=project_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [15]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    project_client.agents.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [16]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    project_client.agents.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(project_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_eHTLK4i4PZSYqB28fjEKLQ4Z> has been deleted
2 - Thread <thread_28CQgFYx7Wm8YYEnSxdrjiXT> has been deleted
Threads deleted: 2



In [17]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=project_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_EA3Z6uHaV9ZQUglmwZ7uQ9vI> has been deleted
2 - Agent <asst_UgeOu2oEaMvjPNiQyalMOdox> has been deleted
Agents deleted: 2



# HIC SUNT LEONES